In [0]:
'''
啟動伺服器樣板
'''
from flask import Flask,request,abort
from linebot import LineBotApi,WebhookHandler
from linebot.exceptions import InvalidSignatureError
import json

secret_file = json.load(open('line_bot_secret_key.json','r'))
server_url = secret_file.get('server_url')
app = Flask(__name__,static_url_path='/image_trpg_elevator',static_folder='../image_trpg_elevator')

line_bot_api = LineBotApi(secret_file.get("channel_access_token"))
handler = WebhookHandler('99f76d3a1e50aa9805bc4362dd5b7766')

@app.route('/',method = ['POST'])
def callback():
    signature = request.headers['X-Link-Signature']
    body = request.get_data(as_text=True)
    app.logger.info('Request body:' + body)

    try:
        handler.handle(body,signature)
    except:
        abort(400)
    return "OK"

In [0]:
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageSendMessage

# 關注時就發消息
reply_message_list = [
ImageSendMessage(original_content_url='https://{}/warning.png'.format(server_url)),
    TextSendMessage(text='本遊戲含有恐怖及克蘇魯神話成分，請斟酌遊玩')
]

In [0]:
if __name__ =='__main__':
    app.run(host='0.0.0.0')